In [ ]:
!pip3 install llama-index --upgrade
!pip3 install llama-index-readers-google
!pip3 install docx2txt
!pip3 install openpyxl
!pip3 install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
!rm token.json

In [ ]:
from venv import logger
from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from llama_index.readers.google import GoogleDriveReader
from llama_index.core import VectorStoreIndex
import os

# set tokens 
SLACK_BOT_TOKEN = ""
SLACK_APP_TOKEN = ""
doc_id = ""

app = App(token=SLACK_BOT_TOKEN)

# Google Drive Authentication and Document Loading
SCOPES = [
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/documents.readonly'
    'https://www.googleapis.com/auth/spreadsheets.readonly',
    'https://www.googleapis.com/auth/drive.readonly'
]

def authenticate_gdrive():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

def load_single_document(doc_id):
    creds = authenticate_gdrive()
    service = build('drive', 'v3', credentials=creds)
    loader = GoogleDriveReader()
    docs = loader.load_data(document_ids=[doc_id]) 
    return docs

def create_index(doc_id):
    google_docs = load_single_document(doc_id)
    index = VectorStoreIndex.from_documents(google_docs)
    return index.as_query_engine()

# Event listener for the Slack bot
@app.event("app_mention") 
def handle_app_mention_events(body, say):
    text = body['event']['text']
    if "search" in text.lower():
        doc_id = ""  # replace with your document ID
        query_engine = create_index(doc_id) # replace as neccessary 
        response = query_engine.query(query)
        say(response.response)  # Assuming response 


if __name__ == "__main__":
    handler = SocketModeHandler(app, SLACK_APP_TOKEN)
    handler.start()